In [ ]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

# Create the CLIPS environment
env = clips.Environment()

import logging
logging.basicConfig(level=10, format='%(message)s')
router = clips.LoggingRouter()
env.add_router(router)

# Define templates
DEFTEMPLATE_PATIENT = """
(deftemplate patient
    (slot name (type STRING))
    (slot has_allergy (type SYMBOL) (allowed-symbols yes no))
    (multislot allergies (type STRING))) ; 'allergies' is a multislot for strings
"""
env.build(DEFTEMPLATE_PATIENT)

DEFTEMPLATE_FOOD = """
(deftemplate food
    (slot name (type STRING))
    (multislot ingredients (type STRING))) ; 'ingredients' is a multislot for strings
"""
env.build(DEFTEMPLATE_FOOD)

DEFTEMPLATE_ALERT = """
(deftemplate alert
    (slot patient (type STRING))
    (slot allergen (type STRING))
    (slot food (type STRING))
    (slot message (type STRING)))
"""
env.build(DEFTEMPLATE_ALERT)

# Corrected Initial facts
DEFFACTS_INITIAL_STATUS = """
(deffacts initial-status
    (patient (name "unknown") (has_allergy no) (allergies)) ; Leave 'allergies' empty
    (food (name "burger") (ingredients "beef" "lettuce" "tomato" "bun" "cheese" "pickles" "onions" "ketchup" "mustard"))
    (food (name "salad") (ingredients "lettuce" "tomato" "cucumber" "dressing" "carrots" "olives" "feta cheese"))
    (food (name "pasta") (ingredients "wheat" "egg" "salt" "olive oil" "garlic" "tomatoes" "parmesan cheese"))
    (food (name "sandwich") (ingredients "bread" "lettuce" "tomato" "cheese" "ham" "turkey" "mayonnaise" "mustard"))
    (food (name "pizza") (ingredients "dough" "tomato sauce" "mozzarella cheese" "pepperoni" "bell peppers" "onions" "olives" "mushrooms"))
    (food (name "soup") (ingredients "chicken broth" "carrots" "celery" "onions" "chicken" "noodles" "salt" "pepper"))
    (food (name "smoothie") (ingredients "banana" "strawberry" "yogurt" "honey" "orange juice" "ice"))
    (food (name "ice cream") (ingredients "milk" "cream" "sugar" "vanilla extract" "chocolate chips"))
    (food (name "cereal") (ingredients "oats" "sugar" "corn syrup" "raisins" "almonds"))
    (food (name "grilled chicken") (ingredients "chicken breast" "olive oil" "lemon" "garlic" "rosemary" "salt" "pepper"))
    (food (name "stir fry") (ingredients "chicken" "broccoli" "bell peppers" "carrots" "soy sauce" "garlic" "ginger"))
    (food (name "tacos") (ingredients "tortilla" "ground beef" "lettuce" "cheese" "salsa" "sour cream" "guacamole"))
    (food (name "pancakes") (ingredients "flour" "milk" "eggs" "sugar" "baking powder" "butter" "maple syrup"))
    (food (name "cupcakes") (ingredients "flour" "sugar" "butter" "eggs" "vanilla extract" "baking powder" "milk" "frosting"))
    (food (name "hot dog") (ingredients "bun" "sausage" "ketchup" "mustard" "onions" "relish"))
    (food (name "chili") (ingredients "ground beef" "tomatoes" "beans" "onions" "chili powder" "garlic"))
    (food (name "omelette") (ingredients "eggs" "milk" "cheese" "spinach" "mushrooms" "salt" "pepper"))
    (food (name "sushi") (ingredients "rice" "nori" "raw fish" "cucumber" "avocado" "soy sauce"))
    (food (name "burrito") (ingredients "tortilla" "rice" "beans" "ground beef" "cheese" "sour cream" "guacamole"))
    (food (name "quiche") (ingredients "eggs" "milk" "cheese" "spinach" "pie crust" "onions"))
    (food (name "lasagna") (ingredients "pasta sheets" "ground beef" "tomato sauce" "cheese" "ricotta" "spinach"))
    (food (name "fried rice") (ingredients "rice" "soy sauce" "egg" "peas" "carrots" "green onions"))
    (food (name "spring rolls") (ingredients "rice paper" "lettuce" "carrots" "cucumber" "shrimp" "peanut sauce"))
    (food (name "meatloaf") (ingredients "ground beef" "bread crumbs" "egg" "onion" "ketchup" "salt" "pepper"))
    (food (name "mac and cheese") (ingredients "pasta" "cheese" "milk" "butter" "flour" "salt"))
    (food (name "shepherd's pie") (ingredients "ground lamb" "mashed potatoes" "peas" "carrots" "onions" "gravy"))
    (food (name "caesar salad") (ingredients "romaine lettuce" "croutons" "parmesan cheese" "caesar dressing" "chicken"))
    (food (name "ratatouille") (ingredients "eggplant" "zucchini" "tomatoes" "bell peppers" "garlic" "olive oil"))
    (food (name "chicken curry") (ingredients "chicken" "coconut milk" "curry powder" "onions" "garlic" "ginger"))
    (food (name "beef stew") (ingredients "beef" "potatoes" "carrots" "celery" "onions" "beef broth" "bay leaves"))
)
"""
env.build(DEFFACTS_INITIAL_STATUS)

# Prompt mapping
prompt_map = {
    "patient:name": "Enter your name: ",
    "patient:has_allergy": "Do you have any allergies? (yes or no): ",
    "patient:allergies": "What kind of allergies do you have? (list space-separated): ",
    "food:name": "Choose a food from the list above. Enter the food name: ",
}

# Build read_assert for interactive inputs
build_read_assert(env, prompt_map)

# Rule to read patient details
DEFRULE_READ_PATIENT_DETAILS = """
(defrule read-patient-details
    =>
    (read_assert patient)
)
"""
env.build(DEFRULE_READ_PATIENT_DETAILS)

# Rule to display all available food items
DEFRULE_DISPLAY_FOOD_LIST = """
(defrule display-food-list
    =>
    (printout t "Here is the list of available foods:" crlf)
    (do-for-all-facts ((?f food)) TRUE
        (printout t "Food Name: " (fact-slot-value ?f name) " | Ingredients: (")
        (foreach ?ingredient (fact-slot-value ?f ingredients)
            (printout t ?ingredient " "))
        (printout t ")" crlf))
)
"""
env.build(DEFRULE_DISPLAY_FOOD_LIST)

# Rule to read food choice and display the list of available foods
DEFRULE_READ_FOOD_CHOICE = """
(defrule read-food-choice
    =>
    (printout t "Here is the list of available foods:" crlf)
    (do-for-all-facts ((?f food)) TRUE
        (printout t "Food Name: " (fact-slot-value ?f name) " | Ingredients: (")
        (foreach ?ingredient (fact-slot-value ?f ingredients)
            (printout t ?ingredient " "))
        (printout t ")" crlf))
    (printout t "Choose a food from the list above. Enter the food name: " crlf)
    (read_assert food)
)
"""
env.build(DEFRULE_READ_FOOD_CHOICE)

# Rule to check food against patient allergies
DEFRULE_CHECK_FOOD = """
(defrule check-food
    (patient (name ?pname) (allergies $?allergies))
    (food (name ?foodname) (ingredients $?ingredients))
    =>
    (bind ?common-allergens (intersection$ ?allergies ?ingredients))
    (if (neq (length$ ?common-allergens) 0)
        then
        (bind ?allergen-str "")
        (foreach ?allergen ?common-allergens
            (bind ?allergen-str (str-cat ?allergen-str ?allergen " ")))
        (assert (alert
            (patient ?pname)
            (allergen ?allergen-str)
            (food ?foodname)
            (message (str-cat "Uh oh, " ?pname "! You can't have this! This food contains: " ?allergen-str))))
        (println "_________________________")
        (println "Alert: Uh oh, " ?pname "! You can't have this! This food contains: " ?allergen-str)
        (println "_________________________"))
    else
        (assert (alert
            (patient ?pname)
            (allergen "None")
            (food ?foodname)
            (message (str-cat "This food does NOT contain any allergen, " ?pname "! Enjoy your food :)")))
        (println "_________________________")
        (println "This food does NOT contain any allergen, " ?pname "! Enjoy your food :)")
        (println "_________________________")))
"""
env.build(DEFRULE_CHECK_FOOD)

# Initialize and run
env.reset()
env.assert_string("(initial-status)")  
env.run()
print("List of available foods displayed.")

# Print final facts to see system state
print_facts(env)

Enter your name:  johanna
Do you have any allergies? (yes or no):  no
What kind of allergies do you have? (list space-separated):  peanut


Here is the list of available foods:
Food Name: burger | Ingredients: (beef lettuce tomato bun cheese pickles onions ketchup mustard )
Food Name: salad | Ingredients: (lettuce tomato cucumber dressing carrots olives feta cheese )
Food Name: pasta | Ingredients: (wheat egg salt olive oil garlic tomatoes parmesan cheese )
Food Name: sandwich | Ingredients: (bread lettuce tomato cheese ham turkey mayonnaise mustard )
Food Name: pizza | Ingredients: (dough tomato sauce mozzarella cheese pepperoni bell peppers onions olives mushrooms )
Food Name: soup | Ingredients: (chicken broth carrots celery onions chicken noodles salt pepper )
Food Name: smoothie | Ingredients: (banana strawberry yogurt honey orange juice ice )
Food Name: ice cream | Ingredients: (milk cream sugar vanilla extract chocolate chips )
Food Name: cereal | Ingredients: (oats sugar corn syrup raisins almonds )
Food Name: grilled chicken | Ingredients: (chicken breast olive oil lemon garlic rosemary salt pepper )
Food Name: st

Invalid inputs provided for patient:
invalid cardinality for slot 'allergies'
